# Sign language project using Tensorflow 

## Goal

This notebook is to build a sign language classifier. One will be able to create own data, label it and train the classifier on specific signs. We will work with the American Sign Language (ASL) codes.


**Import libraries**

In [1]:
import uuid
import numpy as np
import os
import time
from pathlib import Path
import cv2 
import random
import shutil
import matplotlib.pyplot as plt

import tensorflow as tf
from PIL import Image
from io import BytesIO

**Note:** images should have been collected.


### Label collected data

In [4]:
labels = ['hello', 'thanks', 'iloveyou', 'yes', 'no']

In [5]:
for label in labels:
    i = 0
    label_path = os.path.join('workspace1', 'images', 'collectedimages', label)

    for filename in os.listdir(label_path):
        my_dest = label + str(i) + ".jpg"
        my_source = os.path.join(label_path, filename)
        my_dest = os.path.join(label_path, my_dest)
        # rename() function will rename all the files
        os.rename(my_source, my_dest)
        i += 1 

**Split data to train/test folder**

In [6]:
labels = ['hello', 'thanks', 'iloveyou', 'yes', 'no']
source_path = os.path.join('workspace1', 'images')
        
paths = ['train', 'test']
for p in paths:
    for label in labels:
        path = os.path.join(source_path, p, label)
        if not os.path.exists(path):
            !mkdir {path}  

In [7]:
labels = ['hello', 'thanks', 'iloveyou', 'yes', 'no']
number_imgs = 10
test_size = int(np.ceil(0.25*number_imgs))
train_size = number_imgs - test_size

for label in labels:
    train_path = os.path.join('workspace1', 'images', 'train', label)
    test_path = os.path.join( 'workspace1', 'images', 'test', label)

    label_path = os.path.join('workspace1', 'images', 'collectedimages', label)
 
    file_list = os.listdir(label_path)
    
    for file_name in random.sample(file_list, train_size):
        shutil.copy(os.path.join(label_path, file_name), train_path)
    
    for file_name in random.sample(file_list, test_size):
        shutil.copy(os.path.join(label_path, file_name), test_path)


In [8]:
# write labels
labels = [
    {'name':'hello', 'id': 1}, 
    {'name':'iloveyou', 'id': 2},
    {'name':'no', 'id': 3},
    {'name':'thanks', 'id': 4},
    {'name':'yes', 'id': 5}
]

with open('workspace1/annotations/label_map.pbtxt', 'w') as f:
    
    for label in labels:
        f.write('item { \n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')

In [9]:
# Patch the location of gfile
tf.gfile = tf.io.gfile

# convert to npy
def convert_image_into_numpy_array(path):
    '''Puts image into numpy array to feed into tensorflow graph.
    Get a numpy array with shape (img_height, img_width, 3), where channels=3 for RGB.'''

    img_data = tf.io.gfile.GFile(path, 'rb').read()
    image = Image.open(BytesIO(img_data))
    (im_width, im_height) = image.size
    return np.array(image.getdata()).reshape(
        (im_height, im_width, 3)).astype(np.uint8)

In [11]:
hello_train_path = os.path.join('workspace1', 'images', 'train', 'hello','hello3.jpg')
hello3 = convert_image_into_numpy_array(hello_train_path)

In [12]:
hello3.shape

(480, 640, 3)

In [13]:
# plot images
def plotImages(images_arr):
    fig, axes = plt.subplots(1, 10, figsize=(30,20))
    axes = axes.flatten()
    for img, ax in zip(images_arr, axes):
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        ax.imshow(img)
        ax.axis('off')
    plt.tight_layout()
    plt.show()


In [14]:
# plotImages(hello3)
# print(imgs.shape)
# print(labels) 